In [ ]:
from libraries import *
from parameters import *
import statsmodels.api as sm
import sklearn.feature_selection
import statsmodels.api as sm
import sklearn.feature_selection
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')
from statannot import add_stat_annotation
from numpy.random import seed
from numpy.random import randint
import statsmodels.formula.api as smf

In [ ]:
os.getcwd()
os.chdir(projectDir)

In [ ]:
adataC = sc.read('outputs/anndata/adata-hash-features_singlets_SingleKO_08122020_PerGENE.h5ad')


In [ ]:
adataC.X.shape

In [ ]:
covariates = adataC.uns['feature_barcode_names_filtered_GENES']
covariates = covariates[covariates != "GENE_CONTROL_"]

In [ ]:
guideMatrix = adataC.obs[covariates]
guideMatrix.shape
guideMatrix.rename(columns = {'GENE_Rnf8-cmtr1_':'GENE_Rnf8cmtr1_'}, inplace = True) 
guideMatrix.rename(columns = {'GENE_Siah1-ps1_':'GENE_Siah1ps1_'}, inplace = True) 
guideMatrix.rename(columns = {'GENE_Siah1-ps2_':'GENE_Siah1ps2_'}, inplace = True) 


In [ ]:
## Add covariates that should be corrected for
df = adataC.obs[["n_genes", "mt_frac", "leiden" ]]

guideMatrix = guideMatrix.join(df)
guideMatrix.shape

In [ ]:
expressionMatrix = adataC.X

In [ ]:
pd.Series(adataC.var_names).to_csv("/home/beraslan/jovian-work/analysisSingle/Gene_names.csv", index=False)

In [ ]:
allVars = pd.Series(guideMatrix.columns) 
allVars

In [ ]:
all_columns = "+".join(allVars[~allVars.isin(["leiden"])])


In [ ]:
my_formula = "y~" + all_columns
my_formula

In [ ]:
guideMatrix["y"] = np.array(expressionMatrix)[:,1]


In [ ]:
est = smf.mixedlm(my_formula, guideMatrix, groups=guideMatrix["leiden"], missing='drop',re_formula=("~"+all_columns))

In [ ]:
mdf = est.fit(method=["bfgs"])

In [ ]:
mdf.random_effects

In [ ]:
for i in range(5760,6000,20):
    print(i)
    pvalDF = pd.DataFrame()
    coefDF = pd.DataFrame()
    
    for j in range(i,i+20):
        print(j)
        guideMatrix["y"] = np.array(expressionMatrix)[:,j]
        est = smf.mixedlm(my_formula, guideMatrix, groups=guideMatrix["leiden"], missing='drop')
        mdf = est.fit(method=["bfgs"])
        pvalDF[str(j)] = mdf.summary().tables[1]["P>|z|"]
        coefDF[str(j)] = mdf.summary().tables[1]["Coef."]
    
    coefDF.index = mdf.summary().tables[1].index
    pvalDF.index = mdf.summary().tables[1].index
    pvalDF.to_csv("pvalues_"+str(i)+"_"+str(i+20)+"_MixedEffect.csv")   
    coefDF.to_csv("coefs_"+str(i)+"_"+str(i+20)+"_MixedEffec.csv")   